In [1]:
# Data manipulation
import os
import pandas as pd
import csv
import json
import jsonlines
import jsonlines as jl
from pathlib import Path
from dotenv import load_dotenv
from typing import List
import re

# Machine Learning
import torch
import torch.nn as nn
import tensorflow as tf
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt

# Transformers and Langchain
from pydantic import ValidationError, BaseModel, Field

# API and utility
from huggingface_hub import login
from together import Together
import time
from tqdm import tqdm
import accelerate


load_dotenv()

api_key = os.environ.get('TOGETHER_API_KEY')
client = Together(api_key=api_key)

In [2]:
class ArgumentClassification(BaseModel):
    id: str = Field(description="The ID the comment being analyzed")    
    label: int = Field(description="The label associated with the argument (0 or 1)") 

In [3]:
def classify_text(id: str, comment_text: str, argument: str) -> dict:
    extract = client.chat.completions.create(
        messages=[
            {"role": "system", "content": f"""
            Analyze the given comment about gay marriage in relation to a specific argument. You need to:
            Identify if the comment makes use of the given argument. If it does, assign the label 1. If it does not, assign the label 0.
            Do NOT use any other label.
            Do NOT include the comment or the argument in the response.
            
            The argument to analyze is: {argument}
            
            Provide your response in the following JSON format:
            
            {{
                "id": "{id}",
                "label": "the label for the use of the argument in the comment"
            }}
            
            Analyze the following comment in relation to the given argument:
            """},
            {"role": "user", "content": comment_text},
        ],
        model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
        temperature=0,
        top_k=1,
        top_p=1,
        response_format={
            "type": "json_object",
            "schema": ArgumentClassification.model_json_schema(),
        }
    )
    
    return json.loads(extract.choices[0].message.content)


## GAY MARRIAGE

In [4]:
gm = pd.read_csv('../../clean_data/GM_structured.csv')

def process_comments_with_arguments(df: pd.DataFrame, output_file: str):
    with jsonlines.open(output_file, mode='w') as writer:
        for idx, row in tqdm(df.iterrows(), total=len(df), desc="Processing comments"):
            comment_id = row['id']
            comment_text = row['comment_text']
            argument_text = row['argument_text']

            try:
                classification = classify_text(comment_id, comment_text, argument_text)
                writer.write(classification)

            except json.JSONDecodeError as e:
                print(f"JSONDecodeError for comment: {comment_text[:50]}... - Error: {e}")
                continue

            except Exception as e:
                print(f"An unexpected error occurred for comment: {comment_text[:50]}... - Error: {e}")
                continue

process_comments_with_arguments(gm, 'comarg_gm_argument_identification.jsonl')

Processing comments: 100%|██████████| 1285/1285 [28:28<00:00,  1.33s/it]


In [5]:
def classify_text_ugip(id: str, comment_text: str, argument: str) -> dict:
    extract = client.chat.completions.create(
        messages=[
            {"role": "system", "content": f"""
            Analyze the given comment in relation to a specific argument about whether "Under God" should appear in the US Pledge of Allegiance.
            Identify if the comment makes use the given argument. If it does, assign the label 1. If it does not, assign the label 0.
            Do NOT use any other label.
            Do NOT include the comment or the argument in the response.

            The argument to analyze is: {argument}
            
            Provide your response in the following JSON format:
            
            {{
                "id": "{id}",
                "label": "the label for the use of the argument in the comment"
            }}
            
            Analyze the following comment in relation to the given argument:
            """},
            {"role": "user", "content": comment_text},
        ],
        model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
        temperature=0,
        top_k=1,
        top_p=1,
        response_format={
            "type": "json_object",
            "schema": ArgumentClassification.model_json_schema(),
        }
    )
    
    return json.loads(extract.choices[0].message.content)


In [6]:
ugip = pd.read_csv('../../clean_data/UGIP_structured.csv')

def process_comments_with_arguments(df: pd.DataFrame, output_file: str):
    with jsonlines.open(output_file, mode='w') as writer:
        for idx, row in tqdm(df.iterrows(), total=len(df), desc="Processing comments"):
            comment_id = row['id']
            comment_text = row['comment_text']
            argument_text = row['argument_text']

            try:
                classification = classify_text_ugip(comment_id, comment_text, argument_text)
                writer.write(classification)

            except json.JSONDecodeError as e:
                print(f"JSONDecodeError for comment: {comment_text[:50]}... - Error: {e}")
                continue

            except Exception as e:
                print(f"An unexpected error occurred for comment: {comment_text[:50]}... - Error: {e}")
                continue

process_comments_with_arguments(ugip, 'comarg_ugip_argument_identification.jsonl')

Processing comments:  28%|██▊       | 286/1013 [06:19<17:17,  1.43s/it]

An unexpected error occurred for comment: Freedom of religon!!! seriously... - Error: Error code: 400 - {"message": "Bad Request", "type_": "invalid_request_error"}


Processing comments: 100%|██████████| 1013/1013 [22:17<00:00,  1.32s/it]
